In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
from sklearn.model_selection import train_test_split

In [2]:
train_data0 = np.load('data0.npy')
train_lab0 = np.load('lab0.npy')

train_data1 = np.load('data1.npy')
train_lab1 = np.load('lab1.npy')

train_data2 = np.load('data2.npy')
train_lab2 = np.load('lab2.npy')

final_train = np.concatenate((train_data0 , train_data1 , train_data2) , axis=0)
final_labels = np.concatenate((train_lab0,train_lab1,train_lab2), axis=0)




X_train, X_val, y_train, y_val = train_test_split(final_train, final_labels, test_size=0.2, random_state=42)

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [4]:
class MyDataset(Dataset):
    def __init__(self, images, labels):
        self.images = torch.FloatTensor(images.reshape(-1, 1, 40, 168)) / 255.0
        self.labels = torch.FloatTensor(labels)

    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

In [5]:
train_dataset = MyDataset(X_train, y_train)
val_dataset = MyDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [6]:
class BaselineCNN(nn.Module):
    def __init__(self):
        super(BaselineCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
        )
        self.flatten_size = self._get_flatten_size()

        self.fc_layers = nn.Sequential(
            nn.Linear(self.flatten_size, 128),  # have to check for dimensions from conv layers
            nn.ReLU(),
            
            nn.Linear(128, 1)
        )


    

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x
    
    def _get_flatten_size(self):
        
        x = torch.randn(1, 1, 40, 168)
        x = self.conv_layers(x)
        return x.numel()


    # used the above function to get the dimension to put in the fc layers






In [7]:
def train_model(model, train_loader, val_loader, criterion, optimizer, device , epochs=10):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs.squeeze(), labels)
            loss.backward()
            optimizer.step()            
            train_loss += loss.item() * images.size(0)

        train_loss /= len(train_loader.dataset)

        
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs.squeeze(), labels)
                val_loss += loss.item() * images.size(0)
        val_loss /= len(val_loader.dataset)

        print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")




In [8]:
model = BaselineCNN()
criterion = nn.MSELoss()  
optimizer = optim.Adam(model.parameters(), lr=0.001)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
train_model(model, train_loader, val_loader, criterion, optimizer, device , epochs=25)
torch.save(model.state_dict(), 'mymodel.pth')



Epoch 1/25, Train Loss: 29.2277, Val Loss: 19.5779
Epoch 2/25, Train Loss: 18.7371, Val Loss: 15.7543
Epoch 3/25, Train Loss: 13.5813, Val Loss: 13.5064
Epoch 4/25, Train Loss: 11.1815, Val Loss: 12.0491
Epoch 5/25, Train Loss: 9.6474, Val Loss: 10.6231
Epoch 6/25, Train Loss: 8.4935, Val Loss: 11.7011
Epoch 7/25, Train Loss: 7.4176, Val Loss: 10.3683
Epoch 8/25, Train Loss: 6.2711, Val Loss: 10.3242
Epoch 9/25, Train Loss: 5.1426, Val Loss: 12.2667
Epoch 10/25, Train Loss: 4.2357, Val Loss: 11.7438
Epoch 11/25, Train Loss: 3.3875, Val Loss: 10.8554
Epoch 12/25, Train Loss: 2.8062, Val Loss: 11.5806
Epoch 13/25, Train Loss: 2.1337, Val Loss: 11.2999
Epoch 14/25, Train Loss: 1.7141, Val Loss: 11.7414
Epoch 15/25, Train Loss: 1.4720, Val Loss: 11.8763
Epoch 16/25, Train Loss: 1.2141, Val Loss: 11.7956
Epoch 17/25, Train Loss: 1.0427, Val Loss: 12.1995
Epoch 18/25, Train Loss: 0.9531, Val Loss: 12.4429
Epoch 19/25, Train Loss: 0.8774, Val Loss: 12.2605
Epoch 20/25, Train Loss: 0.7668, Val